<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignmnet 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

## Import Boston Housing Dataset

In [33]:
from tensorflow import keras
from tensorflow.keras.datasets import boston_housing
import numpy as np

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [34]:
# concatenate all data into one array for fit/transform, normalization

set1 = np.append(x_train, y_train.reshape(-1,1), 1)
set2 = np.append(x_test, y_test.reshape(-1,1), 1)

data = np.append(set1, set2, 0)

In [39]:
data[:,13]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

## Build normalization pipeline

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler


scaler = RobustScaler()

norm_pipe = Pipeline([
    ('scaler', scaler)
])

In [83]:
from sklearn.model_selection import train_test_split

# Transform data and regenerate X_train, y_train ...
data_transformed = norm_pipe.fit_transform(data)

X = data_transformed[:,0:13]
y = data_transformed[:,13:]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [99]:
X[0][12], y[0][0]

(0.7356321839080457, -0.7523510971786832)

## Define Network

In [111]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(13,input_dim=13, activation='sigmoid'))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse'])
model.fit(X_train,y_train, epochs=1000, verbose=0)

In [117]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 14        
Total params: 196
Trainable params: 196
Non-trainable params: 0
_________________________________________________________________


In [112]:
model.evaluate(X_test, y_test)

102/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[-5.680700414321002, 0.75868195]

In [114]:
test_feat

array([-0.05938742,  6.4       , -0.60263362,  0.        , -0.71428571,
       -0.73915989, -1.13353721,  2.38913624, -0.05      ,  0.01033592,
        1.05357143, -0.41443818, -0.33083458])

In [148]:
import pandas as pd

# Get prediction back out

prediction_set_transformed = model.predict(X_test)

# invert transform to see real value differences
prediction_set = np.append(X_test, prediction_set_transformed, 1)
prediction = norm_pipe.inverse_transform(prediction_set)

# combine targets to review
y_test = set2[:, 13:]
y_pred = prediction[:, 13:]

df = pd.DataFrame(np.append(y_test, y_pred, 1))
df.columns = ['y_test', 'y_pred']
df['diff'] = df.y_test - df.y_pred
df.head()

,y_test,y_pred,diff
0,7.2,19.590203,-12.390203
1,18.8,23.334994,-4.534994
2,19.0,22.605076,-3.605076
3,27.0,33.748345,-6.748345
4,22.2,20.722321,1.477679


**Results** are not fantastic.  I think I'm introducing unnecessary compute complexity in transforming the target with RobustScaler.  The transform -> inverse_transform introduce a lot of potential error and also reduces the extreme differences in price/feature_set seen in this set.  Might be good for constraining targets if targets thesmelves 'should' exist within a certain range.

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

## Import Fashion-MNIST dataset

In [149]:
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [152]:
x_train.shape

(60000, 28, 28)

In [153]:
# input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

## Build Normalization Pipeline

In [156]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

mnist_pipe = Pipeline([
    ('scaler', minmax)
])

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.